<a href="https://colab.research.google.com/github/sezinbiner/ClassificationOfCovidXray/blob/main/generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import cv2
import os

DATADIR = "/content/gdrive/MyDrive/bitirme dataset/test" 
classes = ["normal","covid"] 

X = []
y = []
i = -1
for class_name in classes: 
  i += 1
  print(class_name)
  path = os.path.join(DATADIR,class_name)
  print(path)
  os.chdir(path) 
  images = os.listdir(path)
  for im in images:
      try:
        print(im)
        img = cv2.imread(im, cv2.IMREAD_COLOR) 
        img = cv2.resize(img, (224,224))
        X.append(img) 
        y.append(classes.index(class_name))   
        print(len(X))  
      except Exception as e:
          pass

normal
/content/gdrive/MyDrive/bitirme dataset/test/normal
Normal-6393.png
1
Normal-6397.png
2
Normal-639.png
3
Normal-6412.png
4
Normal-640.png
5
Normal-6406.png
6
Normal-6392.png
7
Normal-6403.png
8
Normal-6396.png
9
Normal-6390.png
10
Normal-6395.png
11
Normal-6402.png
12
Normal-6404.png
13
Normal-6407.png
14
Normal-6394.png
15
Normal-6400.png
16
Normal-6426.png
17
Normal-642.png
18
Normal-641.png
19
Normal-6417.png
20
Normal-6419.png
21
Normal-6435.png
22
Normal-6432.png
23
Normal-6423.png
24
Normal-6418.png
25
Normal-6433.png
26
Normal-6416.png
27
Normal-6421.png
28
Normal-6413.png
29
Normal-6425.png
30
Normal-6427.png
31
Normal-6415.png
32
Normal-6420.png
33
Normal-6431.png
34
Normal-6422.png
35
Normal-643.png
36
Normal-6428.png
37
Normal-6434.png
38
Normal-6429.png
39
Normal-6414.png
40
Normal-6424.png
41
Normal-6430.png
42
Normal-6450.png
43
Normal-6447.png
44
Normal-6445.png
45
Normal-6441.png
46
Normal-6437.png
47
Normal-6448.png
48
Normal-6449.png
49
Normal-6440.png
50
Norma

In [5]:
from keras import layers
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
from keras.models import Sequential
from keras.optimizers import Adam,RMSprop
def build_model(backbone, lr=1e-4):
    model = Sequential()
    model.add(backbone)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    
    model.compile(
        loss='binary_crossentropy',
        optimizer= "adam" ,
        metrics=['accuracy']
    )
    
    return model

In [18]:
BATCH_SIZE =64
from keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(rescale=1./255)

from keras.applications.inception_v3 import  InceptionV3
from keras.applications.vgg16 import VGG16
transfer_model = vgg16(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)



Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [13]:
len(transfer_model.layers)

19

In [19]:
for layer in transfer_model.layers[:17]:
  layer.trainable = False

model = build_model(transfer_model ,lr = 1e-4)
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
global_average_pooling2d_6 ( (None, 512)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 257       
Total params: 14,846,273
Trainable params: 2,491,393
Non-trainable params: 12,354,880
_________________________________________________________________


In [21]:
learn_control = ReduceLROnPlateau(monitor='accuracy', patience=5,
                                  verbose=1,factor=0.2, min_lr=1e-7)

model_checkpoint = ModelCheckpoint('/content/gdrive/My Drive/models/VGG16_tam_boyut.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_dir = "/content/gdrive/MyDrive/bitirme dataset/train"
test_dir = "/content/gdrive/MyDrive/bitirme dataset/test"
validation_dir = "/content/gdrive/MyDrive/bitirme dataset/validation"
class_names = ['covid', 'normal']
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),  
        batch_size=20,
        shuffle=True,
        class_mode='binary')
validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=20,
        shuffle=True,
        class_mode='binary')

Found 4800 images belonging to 2 classes.
Found 1200 images belonging to 2 classes.


In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch = 20,
    epochs=10,
    validation_data = validation_generator,
    validation_steps=25,
    verbose = 1,
    callbacks = [learn_control])

Epoch 1/10
20/20 [==============================] - ETA: 0s - loss: 0.7636 - accuracy: 0.5720

In [ ]:
import numpy as np
x = np.array(X)
y = np.array(y)
x = x/255
from sklearn import metrics
y_pred = model.predict(x)

In [ ]:
probas = np.array(y_pred)
labels = []
for prob in probas:
  if prob < 0.5:
    labels.append(1)
  else:
    labels.append(0)
y_true_labels = y
y_pred_labels = labels

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_true_labels, y_pred_labels)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_true_labels, y_pred_labels,pos_label='positive', average='micro')
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_true_labels, y_pred_labels, average='micro')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_true_labels, y_pred_labels, average='weighted')
print('F1 score: %f' % f1)

In [ ]:
import numpy as np
import keras
x = np.array(X)
y = np.array(y)
x = x/255
reconstructed_model = keras.models.load_model("/content/gdrive/MyDrive/Untitled Folder/my_model.h5")

array([[9.8330235e-01],
       [9.9999946e-01],
       [2.7389753e-01],
       ...,
       [9.5284283e-03],
       [3.3307191e-05],
       [8.6257160e-03]], dtype=float32)

In [ ]:
probas = reconstructed_model.predict(x)
labels = []
for prob in probas:
  if prob < 0.5:
    labels.append(1)
  else:
    labels.append(0)
y_true_labels = y
y_pred_labels = labels